In [26]:
import joblib
EXP_PATH_NAME="SCNN-OCSVM"
RIVAL_PATH_NAME="WACA-OCSVM"
joblib.cpu_count()

32

In [27]:
# !pip install --upgrade pip

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import gc
from joblib import Parallel, delayed

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import dataclasses
from sklearn.svm import OneClassSVM
from dataclasses import asdict
from tqdm import tqdm
import warnings
import random
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold # Feature selector

pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
import seaborn as sns


# Global utitlity functions are in separate notebook
%run ./Classification_utility-functions.ipynb
%run ./SEED-CONSTANTS.ipynb


FINAL_EXP_RESULTS_PATH="clip=False_experiments_results"

np.random.seed(SEED)
print(f"Numpy Seed was set to: {SEED}")

print("Setup Complete")

SEED: 567
FINAL_EXP_RESULTS_PATH: clip=False_experiments_results
--------------------SEED and CONSTANTS imported--------------------
SEED: 567
Python 3.9.10
EER: 0.333, Threshold: 0.600 <-- Arbitrary case
EER: 0.000, Threshold: 0.900 <-- Best case
EER: 1.000, Threshold: 0.900 <-- Worse case
EER: 0.400, Threshold: 0.200 <-- Worse case
EER: 0.167, Threshold: 0.600 <-- Arbitrary case
EER: 0.000, Threshold: 0.900 <-- Best case
EER: 1.000, Threshold: 0.900 <-- Worse case
EER: 0.333, Threshold: 1.000 <-- Worse case
--------------------Utility functions imported--------------------
SEED: 567
FINAL_EXP_RESULTS_PATH: clip=False_experiments_results
--------------------SEED and CONSTANTS imported--------------------
SEED: 567
Python 3.9.10
--------------------Preprocessing utility functions imported--------------------
SEED: 567
FINAL_EXP_RESULTS_PATH: clip=False_experiments_results
--------------------SEED and CONSTANTS imported--------------------
SEED: 567
Python 3.9.10
--------------------Neu

In [3]:
import sklearn
sklearn.__dir__()
sklearn.__version__
WINDOW_SIZE_LST

[125, 250, 500, 750, 1000, 1250, 1500, 1750, 2000]

In [4]:
@dataclasses.dataclass
class ExperimentParameters:
    """Contains all relevant parameters to run an experiment."""

    name: str  # Name of Parameter set. Used as identifier for charts etc.
    frequency: int
    max_subjects: int
    max_test_subjects: int
        
    user_ids: list
    num_sample_points_per_exp: int
    exp_begin_cutoff_idx: int
    exp_end_cutoff_idx: int
        
    
    seconds_per_subject_train: float
    seconds_per_subject_test: float
    window_size: int  # After resampling
    ocsvm_step_width: int  # After resampling
    scaler: str  # StandardScaler, MinMaxScaler, Normalizer, MaxAbsScaler, RobustScaler, PowerTransformer
    scaler_scope: str  # {"subject", "session"}
    scaler_global: bool  # fit transform scale on all data (True) or fit on training only (False)
    ocsvm_kernel: str # ocsvm kernel
    ocsvm_nu: float  # Best value found in random search, used for final model
    ocsvm_gamma: float  # Best value found in random search, used for final model
    feature_cols: list  # Columns used as features
    exclude_subjects: list  # Don't load data from those users
        
    # Calculated values
    def __post_init__(self):
        # HDF key of table:
        self.table_name = f"sensors_{self.frequency}hz"

        

# INSTANCES
# ===========================================================

# NAIVE_APPROACH
# -----------------------------------------------------------
NAIVE_MINMAX_OCSVM = ExperimentParameters(
    name="NAIVE-MINMAX_OCSVM",
    frequency=100,
    max_subjects=29,
    max_test_subjects=10,
    user_ids = [1, 2, 3, 4, 5, 6, 7, 8, 19, 21, 22, 26, 27, 28, 29, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48, 49],
    num_sample_points_per_exp=21000,
    exp_begin_cutoff_idx=500,
    exp_end_cutoff_idx=-500,
    seconds_per_subject_train=210,
    seconds_per_subject_test=210,    
    window_size=250,
    ocsvm_step_width=250,
    scaler="minmax",
    scaler_scope="subject",
    scaler_global=True,
    ocsvm_kernel="rbf",
    ocsvm_nu=None,
    ocsvm_gamma=None,
    feature_cols=[
        "x_a",
        "y_a",
        "z_a",
        "x_g",
        "y_g",
        "z_g",
    ],
    exclude_subjects=[],
)

# VALID_APPROACH
# -----------------------------------------------------------
VALID_MINMAX_OCSVM = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-MINMAX-OCSVM",
    scaler_global=False,
    ocsvm_nu=0.165,
    ocsvm_gamma=0.039,
)

# NAIVE_ROBUST_APPROACH
# -----------------------------------------------------------
NAIVE_ROBUST_OCSVM = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="NAIVE-ROBUST-OCSVM",
    scaler="robust",
    scaler_global=True,
    ocsvm_nu=0.153,
    ocsvm_gamma=0.091,  # below median, selected by chart
)

# ROBUST_APPROACH (VALID)
# -----------------------------------------------------------
VALID_ROBUST_OCSVM_125 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=125
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)


VALID_ROBUST_OCSVM_250 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=250
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_500 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=500
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_750 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=750
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_1000 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=1000
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_1250 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=1250
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_1500 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=1500
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_1750 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=1750
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

VALID_ROBUST_OCSVM_2000 = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-ROBUST-OCSVM",
    scaler="RobustScaler",
    scaler_global=False,
    window_size=2000
#     ocsvm_nu=0.037,
#     ocsvm_gamma= 0.001,
)

# NORMALIZER_APPROACH (VALID)
# -----------------------------------------------------------
VALID_NORMALIZER_OCSVM = dataclasses.replace(
    NAIVE_MINMAX_OCSVM,
    name="VALID-NORMALIZER-OCSVM",
    scaler="Normalizer",
    scaler_global=False,
    ocsvm_nu=0.074,
    ocsvm_gamma= 0.029,
)

In [6]:
P = VALID_ROBUST_OCSVM_2000
P.ocsvm_step_width = int(P.window_size * .5)


P.classifier="OCSVM"
P.ocsvm_kernel = "rbf"
P.train_cores=1 # 20 cores for every user and 1 core for the nested crossval function
P.test_cores=2 # 10 cores for every user and 2 for the nested crossval function


param_dist = {
    "model__gamma": np.logspace(-9, 3), 
    "model__nu": np.linspace(0.0001, 0.1),
}

P.scaler_clip=False

P.is_NN=True


In [7]:
utils_ppp(P)

,Value
name,VALID-ROBUST-OCSVM
frequency,100
max_subjects,29
max_test_subjects,10
user_ids,"[1, 2, 3, 4, 5, 6, 7, 8, 19, 21, 22, 26, 27, 2..."
num_sample_points_per_exp,21000
exp_begin_cutoff_idx,500
exp_end_cutoff_idx,-500
seconds_per_subject_train,210
seconds_per_subject_test,210


In [8]:
utils_eer_scorer

make_scorer(utils_eer, greater_is_better=False)

In [9]:
#include 47 later
# user_ids = [9]
df_exps_dict = load_data_frames(P.user_ids, P.exp_begin_cutoff_idx, P.exp_end_cutoff_idx, P.num_sample_points_per_exp)
raw_dfList_exp1, raw_dfList_exp2 = df_exps_dict['dfList_exp1'], df_exps_dict['dfList_exp2']

Loading exp1 data:
1) accel_count: 28388, gyro_count: 31997
2) accel_count: 26010, gyro_count: 28954
3) accel_count: 28227, gyro_count: 31814
4) accel_count: 24860, gyro_count: 26105
5) accel_count: 24270, gyro_count: 24347
6) accel_count: 25012, gyro_count: 25060
7) accel_count: 25301, gyro_count: 25382
8) accel_count: 21975, gyro_count: 21658
19) accel_count: 24110, gyro_count: 25050
21) accel_count: 24326, gyro_count: 23809
22) accel_count: 29123, gyro_count: 28724
26) accel_count: 23148, gyro_count: 24291
27) accel_count: 24299, gyro_count: 23589
28) accel_count: 23807, gyro_count: 24523
29) accel_count: 24030, gyro_count: 23457
35) accel_count: 24388, gyro_count: 23673
36) accel_count: 24228, gyro_count: 24208
37) accel_count: 31945, gyro_count: 31816
38) accel_count: 22135, gyro_count: 22327
39) accel_count: 23573, gyro_count: 23459
40) accel_count: 23057, gyro_count: 24296
41) accel_count: 24102, gyro_count: 23681
42) accel_count: 24074, gyro_count: 24328
43) accel_count: 22631,

In [10]:
num_sample_points_per_exp_user_47 = 18000
df_exps_dict_user_47 = load_data_frames([47], P.exp_begin_cutoff_idx, P.exp_end_cutoff_idx, num_sample_points_per_exp_user_47)
dfList_exp1_user_47, dfList_exp2_user_47 = df_exps_dict_user_47['dfList_exp1'], df_exps_dict_user_47['dfList_exp2']

raw_dfList_exp1_user_47 = dfList_exp1_user_47
raw_dfList_exp2_user_47 = dfList_exp2_user_47

Loading exp1 data:
47) accel_count: 22777, gyro_count: 22226
Loading exp2 data:
47) accel_count: 17718, gyro_count: 18353


In [11]:
# enable_tf_warnings()


In [12]:
RIVAL_PATH_NAME

'WACA-OCSVM'

# 0. No Smoothing
### Optimizing and Testing

In [30]:
P.smoothing = None

preprocessing_method=None
rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{RIVAL_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

P.cut_off_freq=rival_test_hyperparameters_df["cut_off_freq"][0]
rival_test_hyperparameters_df

,window_size,step_width,Mean_EER,median_nu,median_gamma,Gini_coef,cut_off_freq
0,125,62,0.231879,0.042914,0.655129,0.343750,-99999
1,250,125,0.178423,0.040876,0.655129,0.429536,-99999
2,500,250,0.152316,0.040876,0.372759,0.494137,-99999
3,750,375,0.146990,0.035779,0.120679,0.515640,-99999
4,1000,500,0.133523,0.034759,0.120679,0.592362,-99999
5,1250,625,0.128229,0.034759,0.068665,0.599732,-99999
6,1500,750,0.137243,0.034759,0.030650,0.571571,-99999
7,1750,875,0.108889,0.035779,0.068665,0.651257,-99999
8,2000,1000,0.106611,0.035779,0.039069,0.648584,-99999


In [31]:
init_experiment_params(exp_config=P)
P.cut_off_freq

reseting experiment params successful!


In [35]:
P.scaler_clip

False

In [31]:
# P.variant_suffix="optimal_lr-Max_2000x2"
# P.variant_suffix="optimal_lr-Max_2000x2_tanh_1000_margin=1.0"
# P.variant_suffix="optimal_lr-Max_2000x2_linear_1000_margin=1.0"
# P.variant_suffix="optimal_lr-Max_2000x2_linear_1000_margin=0.5"
# P.variant_suffix="optimal_lr-Max_2000x2_linear_1000_margin=0.1"
# P.variant_suffix="optimal_lr-Max_2000x2_linear_1000_margin=0.25"
# P.variant_suffix="optimal_lr-Max_2000x2_linear_1000_margin=0.75"
# P.variant_suffix="optimal_lr-Max_2000x2_linear_1000_margin=0.5"

In [32]:
rival_test_hyperparameters_df

,window_size,step_width,Mean_EER,median_nu,median_gamma,Gini_coef,cut_off_freq
0,125,62,0.226060,0.035779,0.655129,0.355660,41
1,250,125,0.173420,0.055147,0.372759,0.431020,41
2,500,250,0.144618,0.041895,0.372759,0.499510,41
3,750,375,0.142465,0.034759,0.120679,0.503772,41
4,1000,500,0.125610,0.035779,0.068665,0.566719,41
5,1250,625,0.126424,0.040876,0.068665,0.587064,41
6,1500,750,0.120782,0.033740,0.068665,0.588185,41
7,1750,875,0.104300,0.031701,0.068665,0.656070,41
8,2000,1000,0.095222,0.034759,0.068665,0.669817,41


# 0. No Smoothing
### Optimizing and Testing

In [34]:
init_experiment_params(exp_config=P)

exp_deep_archi_extractors_dict=get_exp_deep_feature_extractors_dict(exp_num=EXP_NUM_0, model_architectures=["SCNN_3_1_conv_1_dense_arg_dict_default"], 
                                                                    variant_suffix=P.variant_suffix)
# exp_deep_archi_extractors_dict=get_exp_deep_feature_extractors_dict(EXP_NUM_0, model_architectures= [list(TRAINING_CONFIG_DICT['1500'].keys())[0]])

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = None

preprocessing_method=None
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/clip={P.scaler_clip}-{P.smoothing}-Prep={preprocessing_method}-variant_suffix={P.variant_suffix}"+\
                "_df_test_dict.txt"
    
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")


EER_df_test_dict={}
for scnn_archi_name, exp_deep_feature_extractors_dict in exp_deep_archi_extractors_dict.items():

    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{RIVAL_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

    # P.cut_off_freq=rival_test_hyperparameters_df["cut_off_freq"][0]
    # print(f"cut_off_freq: {P.cut_off_freq}")

    P.filter_order=4
    P.sampling_freq=100
    P.filtfilt=1
    P.cut_off_freq=30
    ##butter
    ffted_dfList_exp1 = get_ffted_dfList(raw_dfList_exp1, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    ffted_dfList_exp1_user_47 = get_ffted_dfList(raw_dfList_exp1_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
        
    dfList_dict={
                "dfList_exp1": ffted_dfList_exp1,#raw_dfList_exp1,
                "dfList_exp2": ffted_dfList_exp2,#raw_dfList_exp2,
                "dfList_exp1_user_47": ffted_dfList_exp1_user_47,#raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": ffted_dfList_exp2_user_47,#raw_dfList_exp2_user_47
    }
    
    test_dict_key=scnn_archi_name
    EER_df_test_dict[test_dict_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST,
                                                                                             test_set=test_set, exp_config=P, 
                                                                                             extract_features_func_dict=exp_deep_feature_extractors_dict, 
                                                                                             overlap=OVERLAP, 
                                                                                             best_param_df=rival_test_hyperparameters_df)
    with open(test_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"test_dict_key: {test_dict_key}\n")
        f.write(EER_df_test_dict[test_dict_key].to_string())
        
    
#-------
#-------
key_column= ["scnn_archi_name"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')

reseting experiment params successful!
clip=False_experiments_results/SCNN_trained_models/SCNN-OCSVM/0/SCNN_3_1_conv_1_dense_arg_dict_default/1000/SCNN-OCSVM-0-SCNN_3_1_conv_1_dense_arg_dict_default-1000-deep_feature_extractor-optimal_lr-Max_2000x2_linear_1000_margin=0.5
train_set: [7, 24, 8, 11, 13, 19, 28, 21, 26, 3, 20, 22, 6, 25, 16, 1, 17, 27, 5, 47]
test_set: [0, 12, 14, 9, 18, 23, 2, 15, 10, 4]


  0%|          | 0/1 [00:00<?, ?it/s]

window_size: 1000
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
[-1. -1. -1. -1. -1. -1.]
[1.0000002 1.        1.        1.        1.        1.       ]
[-1. -1. -1. -1. -1. -1.]
[0.99999994 1.0000002  0.9999999  1.0000001  1.         1.0000001 ]
[-1. -1. -1. -1. -1. -1.]
[1.         0.9999999  1.0000001  1.         0.99999994 1.        ]
[-1. -1. -1. -1. -1. -1.]
[1.        1.        1.        0.9999999 1.        0.9999999]
[-1. -1. -1. -1. -1. -1.]
[1.        1.0000002 1.        1.        1.        0.9999999]
[-1. -1. -1. -1. -1. -1.]
[1.0000001 0.9999998 0.9999998 0.9999999 1.        1.       ]
[-1. -1. -1. -1. -1. -1.]
[1.         0.99999994 0.99999994 1.         1.         0.9999999 ]
[-1. -1. -1. -1. -1. -1.]
[1.         0.9999999  1.         1.0000001  1.         0.99999994]
[-1. -1. -1. -1. -1. -1.]
[0.9999999  0.9999999  0.99999994 0.99999994 0.99999994 0.9999999 ]
[-1. -1. -1. -1. -1. -1.]
[1.         1.         1.         1.0000001  1.         0.99999994]
Make


  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 8393.64it/s]
/tmp/ipykernel_558858/1897070050.py:65: RuntimeWarning: invalid value encountered in double_scalars
/tmp/ipykernel_558858/1897070050.py:65: RuntimeWarning: invalid value encountered in double_scalars

 20%|██        | 1/5 [00:00<00:00,  4.48it/s]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 5862.06it/s]
/tmp/ipykernel_558858/1897070050.py:65: RuntimeWarning: invalid value encountered in double_scalars
/tmp/ipykernel_558858/1897070050.py:65: RuntimeWarning: invalid value encountered in double_scalars

 40%|████      | 2/5 [00:00<00:00,  4.70it/s]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 14548.40it/s]
/tmp/ipykernel_558858/1897070050.py:65: RuntimeWarning: invalid value encountered in double_scalars
/tmp/ipykernel_558858/1897070050.py:65: RuntimeWarning: invalid value encountered in double_scalars

 60%|██████    | 3/5 [00:00<00:00,  4.95it/s]

run: 3: 100%|██████████| 10/10 [00:00<00:

In [ ]:
# exp_deep_archi_extractors_dict.items()

# 1. Butterworth frequency Cut-off

## 1.1 Naive Approach
### Optimizing and Testing

In [ ]:
init_experiment_params(exp_config=P)

exp_deep_archi_extractors_dict=get_exp_deep_feature_extractors_dict(exp_num=EXP_NUM_1_1)
# exp_deep_archi_extractors_dict=get_exp_deep_feature_extractors_dict(EXP_NUM_1_1, model_architectures= [list(TRAINING_CONFIG_DICT['1000'].keys())[0]])

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butterworth"


preprocessing_method="Naive"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
    
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")
    




EER_df_test_dict={}

for scnn_archi_name, exp_deep_feature_extractors_dict in exp_deep_archi_extractors_dict.items():

    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{RIVAL_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

    P.cut_off_freq=rival_test_hyperparameters_df["cut_off_freq"][0]
    print(f"cut_off_freq: {P.cut_off_freq}")

    ffted_dfList_exp1 = get_ffted_dfList(raw_dfList_exp1, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    ffted_dfList_exp1_user_47 = get_ffted_dfList(raw_dfList_exp1_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)


    dfList_dict={
                "dfList_exp1": ffted_dfList_exp1,
                "dfList_exp2": ffted_dfList_exp2,
                "dfList_exp1_user_47": ffted_dfList_exp1_user_47,
                "dfList_exp2_user_47": ffted_dfList_exp2_user_47
    }



    test_dict_key=scnn_archi_name, P.cut_off_freq
    EER_df_test_dict[test_dict_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, 
                                                                                             test_set=test_set, exp_config=P, 
                                                                                             extract_features_func_dict=exp_deep_feature_extractors_dict, 
                                                                                             overlap=OVERLAP, best_param_df=rival_test_hyperparameters_df)
    with open(test_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"cut_off_freq: {test_dict_key}\n")
        f.write(EER_df_test_dict[test_dict_key].to_string())
        
    
#-------
#-------
key_column= ["scnn_archi_name", "cut_off_freq"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')

In [ ]:
key_column= test_dict_key#["cut_off_freq"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')

In [ ]:
exp_deep_archi_extractors_dict.keys()

In [ ]:
K.clear_session()
tf.compat.v1.reset_default_graph()

In [ ]:
window_size_lst=WINDOW_SIZE_LST
exp_config=P
extract_features_func_dict=exp_deep_archi_extractors_dict["SCNN_1_3_conv_1_dense_arg_dict_default"]
dfList_exp1=dfList_dict["dfList_exp1"]
dfList_exp2=dfList_dict["dfList_exp2"]
EER_distro_df_dict = {}

if exp_config.classifier=="OCSVM":
    columns = ['window_size', "step_width", "Mean_EER", "median_nu", "median_gamma"]

elif exp_config.classifier=="IF":
    columns = ['window_size', "step_width", "Mean_EER", "median_n_estimators", "median_max_samples", "median_contamination", "median_max_features"]

elif exp_config.classifier=="LOF":
    columns = ['window_size', "step_width", "Mean_EER", "median_n_neighbors", "median_contamination"]

else:
    raise Exception("Unspecified classifier")

Mean_EER_df = pd.DataFrame(columns = columns)
for window_size in tqdm(window_size_lst):
    print(f"window_size: {window_size}")
    step_width = int(window_size * (1-overlap))


    X_test_exp1_dict, X_test_exp2_dict, fitted_scaler_test_exp2_dict=get_raw_windows(dfList_exp1, dfList_exp2, window_size, step_width, test_set, exp_config.scaler, 
                                                                                     exp_config.num_sample_points_per_exp, EMA_per_win_span=exp_config.EMA_per_win_span, 
                                                                                     SMA_per_win_winsize=exp_config.SMA_per_win_winsize,
                                                                                     Butter_per_win_argdict=exp_config.Butter_per_win_argdict, 
                                                                                     verbose=0)


    extract_features_func=extract_features_func_dict[window_size]

    X_exp_test_dic = extract_features_func(X_test_exp1_dict, X_test_exp2_dict, fitted_scaler_test_exp2_dict)
    
X_exp_test_dic


In [ ]:
X_test_exp1_dict[0][0].shape

In [ ]:
X_test_exp1_dict.keys()

In [ ]:
#X_test_exp1_dict

In [ ]:
base_path=f"final_experiments_results/{SAVE_PATH_NAME}/SCNN-OCSVM/{EXP_NUM_1_1}"
archi_name = list(TRAINING_CONFIG_DICT['2000'].keys())[0]
# archi_name="SCNN_3_1_conv_1_dense_arg_dict_default"
path_to_archi_name=base_path+f"/{archi_name}"
archi_win_dict={}
final_path=path_to_archi_name+f"/{1750}"
final_path

In [ ]:
loss_func = lambda y_true, dist: k_contrastive_loss(y_true, dist, arg_dict["contrastive_loss_margin"])

In [ ]:
deep_feature_extractor_path=final_path+f"/SCNN-OCSVM-{EXP_NUM_1_1}-{archi_name}-{1750}-deep_feature_extractor"
print(deep_feature_extractor_path)

deep_feature_model=load_model(deep_feature_extractor_path)# custom_objects={'loss_func': loss_func})

In [ ]:
deep_feature_model.predict(np.array([X_test_exp1_dict[0][0]]))

In [ ]:
ExtractDeepFeatures([X_test_exp1_dict], deep_feature_model)

In [ ]:
extract_learnt_features(X_test_exp1_dict, X_test_exp2_dict, fitted_scaler_test_exp2_dict, deep_feature_model, verbose=1)

In [ ]:
MakeDeepXExpDic(X_test_exp1_dict, X_test_exp2_dict, fitted_scaler_test_exp2_dict, 
                    X_test_exp1_dict, X_test_exp2_dict, fitted_scaler_test_exp2_dict, deep_feature_model)


In [ ]:
P.smoothing=None
preprocessing_method=None
df_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_eer_per_window_size_col_df.json"
df = pd.read_json(df_file_name)
df

In [ ]:
P.smoothing="Butterworth"
preprocessing_method="Naive"
df_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_eer_per_window_size_col_df.json"
df = pd.read_json(df_file_name)
df

In [ ]:
P.smoothing="EMA"
preprocessing_method="Naive"
df_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_eer_per_window_size_col_df.json"
df = pd.read_json(df_file_name)
df

In [ ]:
P.smoothing="Butter+EMA"
preprocessing_method="Naive"
df_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_eer_per_window_size_col_df.json"
df = pd.read_json(df_file_name)
df

## 1.2 Realworld Approach (per unknown window application of filter)
### Optimizing and Testing

In [ ]:
init_experiment_params(exp_config=P)
P.smoothing = "Butterworth"

preprocessing_method="Realworld-per_unknown_window"

time_of_execution = time.strftime("%Y-%m-%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/clip={P.scaler_clip}-Smoothing={P.smoothing}-Prep={preprocessing_method}-EER_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/clip={P.scaler_clip}-Smoothing={P.smoothing}-Prep={preprocessing_method}-EER_df_test_dict.txt"

with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{train_file_name}" + "-"*40 + "\n")
    
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")

clf_type="standalone"


time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")



for cv_fold_idx in range(len(THREE_FOLD_CV)):
    
    train_set, test_set=THREE_FOLD_CV[cv_fold_idx]
    print(f"train_set: {train_set}")
    print(f"test_set: {test_set}")
    
    #--------------butter33----------------------
    P.cut_off_freq=33

    P.Butter_per_win_argdict={
        "filter_order": P.filter_order,
        "cut_off_freq": P.cut_off_freq,
        "sampling_freq": P.sampling_freq,
        "filtfilt": P.filtfilt,
        }   


    print(f"cut_off_freq: {P.cut_off_freq}")

    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)


    #--------------butter33----------------------

    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": ffted_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": ffted_dfList_exp2_user_47
    }
        
        

    # save_info_dict={"model_classifier_type": f"SCNN-{clf_type}", "exp_num": EXP_NUM_1_2, "cv_fold_idx": cv_fold_idx}
    # model_training_with_training_config_dict(dfList_dict=dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, overlap=OVERLAP, 
    #                                          training_config_dict=get_training_config_dict('Butter', cv_fold_idx), save_info_dict=save_info_dict)


    exp_deep_archi_extractors_dict=get_exp_deep_feature_extractors_dict(exp_num=EXP_NUM_1_2, 
                                                                        model_architectures=list(TRAINING_CONFIG_CV_DICT["Butter"][cv_fold_idx]['1000'].keys()), 
                                                                        exp_path_name=EXP_PATH_NAME,
                                                                        window_size_lst=WINDOW_SIZE_LST_NN,
                                                                        variant_suffix=f"Butter33-cv{cv_fold_idx}")
        

    key_column= ["cut_off_freq"]
    # -----CV_FOLD-------
    process_cv_fold_OCSVM_LOF_IF_NN(\
                                    cv_fold_idx=cv_fold_idx, 
                                    cv_sets=THREE_FOLD_CV, 
                                    dfList_dict=dfList_dict, 
                                    window_size_lst=WINDOW_SIZE_LST_NN, 
                                    exp_config=P, 
                                    exp_deep_archi_extractors_dict=exp_deep_archi_extractors_dict, 
                                    overlap=OVERLAP, 
                                    param_dist=param_dist, 
                                    train_file_name=train_file_name, 
                                    test_file_name=test_file_name, 
                                    preprocessing_params=P.cut_off_freq,
                                    key_column=key_column,
                                   )
    
    
    del exp_deep_archi_extractors_dict
    gc.collect()

reseting experiment params successful!
train_set: {0, 1, 2, 3, 4, 5, 6, 9, 10, 12, 14, 15, 16, 18, 19, 22, 23, 24, 25, 28}
test_set: {7, 8, 11, 13, 17, 20, 21, 26, 27, 29}
cut_off_freq: 33
clip=False_experiments_results/SCNN_trained_models/SCNN-standalone/1.2/SCNN_3_1_conv_1_dense_arg_dict_default/125/SCNN-standalone-1.2-SCNN_3_1_conv_1_dense_arg_dict_default-125-deep_feature_extractor-Butter33-cv0


2023-08-03 05:23:21.407764: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-08-03 05:23:21.407883: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:163] no NVIDIA GPU device is present: /dev/nvidia0 does not exist
2023-08-03 05:23:21.408730: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


clip=False_experiments_results/SCNN_trained_models/SCNN-standalone/1.2/SCNN_3_1_conv_1_dense_arg_dict_default/250/SCNN-standalone-1.2-SCNN_3_1_conv_1_dense_arg_dict_default-250-deep_feature_extractor-Butter33-cv0
clip=False_experiments_results/SCNN_trained_models/SCNN-standalone/1.2/SCNN_3_1_conv_1_dense_arg_dict_default/500/SCNN-standalone-1.2-SCNN_3_1_conv_1_dense_arg_dict_default-500-deep_feature_extractor-Butter33-cv0
clip=False_experiments_results/SCNN_trained_models/SCNN-standalone/1.2/SCNN_3_1_conv_1_dense_arg_dict_default/750/SCNN-standalone-1.2-SCNN_3_1_conv_1_dense_arg_dict_default-750-deep_feature_extractor-Butter33-cv0
clip=False_experiments_results/SCNN_trained_models/SCNN-standalone/1.2/SCNN_3_1_conv_1_dense_arg_dict_default/1000/SCNN-standalone-1.2-SCNN_3_1_conv_1_dense_arg_dict_default-1000-deep_feature_extractor-Butter33-cv0
clip=False_experiments_results/SCNN_trained_models/SCNN-standalone/1.2/SCNN_3_2_conv_1_dense_arg_dict_default/125/SCNN-standalone-1.2-SCNN_3_2_con

  0%|          | 0/5 [00:00<?, ?it/s]

window_size: 125
NN preprocessing
len(exp1_df_user_set_dict): 20
len(exp2_df_user_set_dict): 20
[-1. -1. -1. -1. -1. -1.]
[0.99999994 1.         1.0000002  0.99999994 1.         0.9999999 ]
[-1. -1. -1. -1. -1. -1.]
[0.9999999  0.99999994 0.99999994 1.         1.         1.        ]
[-1. -1. -1. -1. -1. -1.]
[1.0000001  1.         0.9999998  0.99999994 1.         1.        ]
[-1. -1. -1. -1. -1. -1.]
[1.        1.        1.        1.0000001 0.9999998 1.       ]
[-1. -1. -1. -1. -1. -1.]
[0.9999999  0.9999999  1.0000001  0.9999999  1.         0.99999994]
[-1. -1. -1. -1. -1. -1.]
[1.         0.99999994 0.99999994 1.         0.99999994 0.9999999 ]
[-1. -1. -1. -1. -1. -1.]
[1.         1.0000001  1.0000002  0.99999994 0.99999994 1.0000001 ]
[-1. -1. -1. -1. -1. -1.]
[1.0000001 0.9999998 0.9999998 1.0000001 1.        0.9999998]
[-1. -1. -1. -1. -1. -1.]
[1.         0.9999998  1.         0.9999998  0.99999994 1.        ]
[-1. -1. -1. -1. -1. -1.]
[1.0000001  0.9999999  0.99999994 1.        

...

window_size: 1000
NN preprocessing
len(exp1_df_user_set_dict): 10
len(exp2_df_user_set_dict): 10
[-1. -1. -1. -1. -1. -1.]
[0.9999999  0.99999994 0.99999994 1.         1.         1.        ]
[-1. -1. -1. -1. -1. -1.]
[1.        1.        1.        1.0000001 0.9999998 1.       ]
[-1. -1. -1. -1. -1. -1.]
[1.         0.99999994 0.99999994 1.         0.99999994 0.9999999 ]
[-1. -1. -1. -1. -1. -1.]
[1.         1.0000001  1.0000002  0.99999994 0.99999994 1.0000001 ]
[-1. -1. -1. -1. -1. -1.]
[1.0000001  0.99999994 0.99999994 1.         0.9999998  0.99999994]
[-1. -1. -1. -1. -1. -1.]
[1.        1.        1.0000001 0.9999999 0.9999999 1.       ]
[-1. -1. -1. -1. -1. -1.]
[1.         0.9999999  1.0000001  0.99999994 1.         0.99999994]
[-1. -1. -1. -1. -1. -1.]
[0.99999994 0.9999998  1.         0.9999999  0.99999994 1.0000001 ]
[-1. -1. -1. -1. -1. -1.]
[0.9999999 1.        1.0000001 1.        1.        0.9999999]
[-1. -1. -1. -1. -1. -1.]
[1.         1.         1.         1.0000001  0.99


  0%|          | 0/5 [00:00<?, ?it/s]

run: 0: 100%|██████████| 10/10 [00:00<00:00, 7575.05it/s]

 20%|██        | 1/5 [00:00<00:02,  1.63it/s]

run: 1: 100%|██████████| 10/10 [00:00<00:00, 11472.39it/s]

 40%|████      | 2/5 [00:01<00:01,  1.72it/s]

run: 2: 100%|██████████| 10/10 [00:00<00:00, 5638.26it/s]

 60%|██████    | 3/5 [00:01<00:00,  2.00it/s]

run: 3: 100%|██████████| 10/10 [00:00<00:00, 5342.38it/s]

 80%|████████  | 4/5 [00:01<00:00,  2.15it/s]

run: 4: 100%|██████████| 10/10 [00:00<00:00, 6526.07it/s]

100%|██████████| 5/5 [06:44<00:00, 80.94s/it]


{('SCNN_3_1_conv_1_dense_arg_dict_default', 33):   window_size step_width  Mean_EER median_nu median_gamma Gini_coef
0         125         62  0.145058  0.056166     0.655129   0.60818
0         250        125  0.110253  0.038837     1.151395  0.728172
0         500        250  0.079317   0.06738     0.655129   0.78863
0         750        375  0.118727  0.042914     0.903262  0.753168
0        1000        500  0.049729  0.035779     0.008372  0.902555, ('SCNN_3_2_conv_1_dense_arg_dict_default', 33):   window_size step_width  Mean_EER median_nu median_gamma Gini_coef
0         125         62  0.183238  0.024565     0.012649  0.583838
0         250        125   0.19159  0.036798     0.120679  0.583001
0         500        250  0.117657  0.035779      2.02359   0.65087
0         750        375  0.115939  0.043934     0.655129   0.71121
0        1000        500  0.102493  0.043934     0.001828  0.773218, ('SCNN_1_3_conv_1_dense_arg_dict_default', 33):   window_size step_width  Mean_EER me

  0%|          | 0/5 [00:00<?, ?it/s]

window_size: 125
NN preprocessing
len(exp1_df_user_set_dict): 19
len(exp2_df_user_set_dict): 19


In [62]:
list(TRAINING_CONFIG_CV_DICT["Butter"][cv_fold_idx]['1000'].keys())[:1]

['SCNN_3_1_conv_1_dense_arg_dict_default']

In [ ]:
init_experiment_params(exp_config=P)

exp_deep_archi_extractors_dict=get_exp_deep_feature_extractors_dict(exp_num=EXP_NUM_1_2)
print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butterworth"



preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
    
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")
    



P.Butter_per_win_argdict={
    "filter_order": P.filter_order,
    "cut_off_freq": None,
    "sampling_freq": P.sampling_freq,
    "filtfilt": P.filtfilt,
    }


EER_df_test_dict={}

for scnn_archi_name, exp_deep_feature_extractors_dict in exp_deep_archi_extractors_dict.items():
    
    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{RIVAL_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)
    
    P.cut_off_freq=rival_test_hyperparameters_df["cut_off_freq"][0]
    
    P.Butter_per_win_argdict["cut_off_freq"]=P.cut_off_freq
    print(f"cut_off_freq: {P.cut_off_freq}")

    
    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    
    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": ffted_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": ffted_dfList_exp2_user_47
    }
    

    test_dict_key=scnn_archi_name, P.cut_off_freq
    EER_df_test_dict[test_dict_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, 
                                                                                             exp_config=P, extract_features_func_dict=exp_deep_feature_extractors_dict,  
                                                                                             overlap=OVERLAP, best_param_df=rival_test_hyperparameters_df)
    with open(test_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\Top smoothing parameter/s: {test_dict_key}\n")
        f.write(EER_df_test_dict[test_dict_key].to_string())
        
#-------
#-------
key_column= ["scnn_archi_name", "cut_off_freq"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

# 2. Butterworth frequency Cut-off + EMA span
## 2.1 Naive Approach
### Optimizing and Testing

In [ ]:
init_experiment_params(exp_config=P, exp_num=EXP_NUM_2)

exp_deep_archi_extractors_dict=get_exp_deep_feature_extractors_dict(exp_num=EXP_NUM_2_1)
print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butter+EMA"



preprocessing_method="Naive"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"

with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")
    


EER_df_test_dict={}

for scnn_archi_name, exp_deep_feature_extractors_dict in exp_deep_archi_extractors_dict.items():
    
    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{RIVAL_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

    P.cut_off_freq=rival_test_hyperparameters_df["cut_off_freq"][0]
    P.span=rival_test_hyperparameters_df["EMA_span"][0]

    print(f"cut_off_freq: {P.cut_off_freq}, EMA span: {P.span}")

    
    ffted_dfList_exp1 = get_ffted_dfList(raw_dfList_exp1, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    EMAed_dfList_exp1 = get_EMAed_dfList(ffted_dfList_exp1, span=P.span)
    EMAed_dfList_exp2 = get_EMAed_dfList(ffted_dfList_exp2, span=P.span)
    
    ffted_dfList_exp1_user_47 = get_ffted_dfList(raw_dfList_exp1_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    EMAed_dfList_exp1_user_47 = get_EMAed_dfList(ffted_dfList_exp1_user_47, span=P.span)
    EMAed_dfList_exp2_user_47 = get_EMAed_dfList(ffted_dfList_exp2_user_47, span=P.span)
    
    dfList_dict={
                "dfList_exp1": EMAed_dfList_exp1,
                "dfList_exp2": EMAed_dfList_exp2,
                "dfList_exp1_user_47": EMAed_dfList_exp1_user_47,
                "dfList_exp2_user_47": EMAed_dfList_exp2_user_47
    }
    
    test_dict_key=scnn_archi_name, P.cut_off_freq, P.span
    EER_df_test_dict[test_dict_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, 
                                                                                             exp_config=P, extract_features_func_dict=exp_deep_feature_extractors_dict,
                                                                                             overlap=OVERLAP, best_param_df=rival_test_hyperparameters_df)
    with open(test_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\Top smoothing parameter/s: {test_dict_key}\n")
        f.write(EER_df_test_dict[test_dict_key].to_string())
#-------  
#-------
key_column= ["scnn_archi_name", "cut_off_freq", "EMA_span"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

## 2.2 Realworld Approach (per unknown window application of filter)
### Optimizing and Testing

In [ ]:
init_experiment_params(exp_config=P, exp_num=EXP_NUM_2)
P.smoothing = "Butter+EMA"

preprocessing_method="Realworld-per_unknown_window"

time_of_execution = time.strftime("%Y-%m-%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/clip={P.scaler_clip}-Smoothing={P.smoothing}-Prep={preprocessing_method}-EER_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/clip={P.scaler_clip}-Smoothing={P.smoothing}-Prep={preprocessing_method}-EER_df_test_dict.txt"

with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{train_file_name}" + "-"*40 + "\n")
    
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")

clf_type="standalone"


time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")



for cv_fold_idx in range(len(THREE_FOLD_CV)):
    
    train_set, test_set=THREE_FOLD_CV[cv_fold_idx]
    print(f"train_set: {train_set}")
    print(f"test_set: {test_set}")
    
    #--------------butter33-EMA20----------------------

    P.cut_off_freq=33
    P.span=20

    P.Butter_per_win_argdict={
        "filter_order": P.filter_order,
        "cut_off_freq": P.cut_off_freq,
        "sampling_freq": P.sampling_freq,
        "filtfilt": P.filtfilt,
        }   

    P.EMA_per_win_span=P.span

    print(f"cut_off_freq: {P.cut_off_freq}, EMA span: {P.span}")


    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    EMAed_dfList_exp2 = get_EMAed_dfList(ffted_dfList_exp2, span=P.span)

    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    EMAed_dfList_exp2_user_47 = get_EMAed_dfList(ffted_dfList_exp2_user_47, span=P.span)

    #--------------butter33-EMA20----------------------

    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": EMAed_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": EMAed_dfList_exp2_user_47
    }
        



    exp_deep_archi_extractors_dict=get_exp_deep_feature_extractors_dict(exp_num=EXP_NUM_2_2, 
                                                                        model_architectures=list(TRAINING_CONFIG_CV_DICT["Butter-EMA"][cv_fold_idx]['1000'].keys()), 
                                                                        exp_path_name=EXP_PATH_NAME,
                                                                        window_size_lst=WINDOW_SIZE_LST_NN,
                                                                        variant_suffix=f"Butter33-EMA20-cv{cv_fold_idx}")
        

    key_column= ["cut_off_freq", "EMA_span"]
    # -----CV_FOLD-------
    # for cv_fold_idx in range(len(THREE_FOLD_CV)):
    process_cv_fold_OCSVM_LOF_IF_NN(\
                                    cv_fold_idx=cv_fold_idx, 
                                    cv_sets=THREE_FOLD_CV, 
                                    dfList_dict=dfList_dict, 
                                    window_size_lst=WINDOW_SIZE_LST_NN, 
                                    exp_config=P, 
                                    exp_deep_archi_extractors_dict=exp_deep_archi_extractors_dict, 
                                    overlap=OVERLAP, 
                                    param_dist=param_dist, 
                                    train_file_name=train_file_name, 
                                    test_file_name=test_file_name, 
                                    preprocessing_params=(P.cut_off_freq, P.EMA_per_win_span),
                                    key_column=key_column,
                                   )
    
    
    del exp_deep_archi_extractors_dict
    gc.collect()

In [ ]:
init_experiment_params(exp_config=P, exp_num=EXP_NUM_2)

exp_deep_archi_extractors_dict=get_exp_deep_feature_extractors_dict(exp_num=EXP_NUM_2_2)
print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butter+EMA"



preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"

with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")





P.Butter_per_win_argdict={
    "filter_order": P.filter_order,
    "cut_off_freq": None,
    "sampling_freq": P.sampling_freq,
    "filtfilt": P.filtfilt,
    }


EER_df_test_dict={}

for key_pair in tqdm(cut_off_span_pairs):
    key_pair = tuple(key_pair)
    cut_off_freq, span = key_pair[0], key_pair[1]
    P.cut_off_freq=cut_off_freq
    P.Butter_per_win_argdict["cut_off_freq"]=cut_off_freq
    
    
    P.span=span
    P.EMA_per_win_span=span

    print(f"cut_off_freq: {P.cut_off_freq}")
    print(f"span: {P.span}")

    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    EMAed_dfList_exp2 = get_EMAed_dfList(ffted_dfList_exp2, span=P.span)
    
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    EMAed_dfList_exp2_user_47 = get_EMAed_dfList(ffted_dfList_exp2_user_47, span=P.span)
    
    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": EMAed_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": EMAed_dfList_exp2_user_47
    }
    

    EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                       extract_features_func_dict=exp_deep_feature_extractors_dict,  overlap=OVERLAP, 
                                                                                       best_param_df=EER_df_train_dict[min_key])
    with open(test_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\Top smoothing parameter/s: {min_key}\n")
        f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["cut_off_freq", "EMA_span"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

# 3. EMA span
## 3.1 Naive Approach
### Optimizing and Testing

In [ ]:
init_experiment_params(exp_config=P)

exp_deep_archi_extractors_dict=get_exp_deep_feature_extractors_dict(exp_num=EXP_NUM_3_1)
print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "EMA"


preprocessing_method="Naive"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
    
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")
    
    

EER_df_test_dict={}
for scnn_archi_name, exp_deep_feature_extractors_dict in exp_deep_archi_extractors_dict.items():
    
    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{RIVAL_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

    P.span=rival_test_hyperparameters_df["EMA_span"][0]
    print(f"EMA span: {P.span}")

    
    EMAed_dfList_exp1 = get_EMAed_dfList(raw_dfList_exp1, span=P.span)
    EMAed_dfList_exp2 = get_EMAed_dfList(raw_dfList_exp2, span=P.span)
    
    EMAed_dfList_exp1_user_47 = get_EMAed_dfList(raw_dfList_exp1_user_47, span=P.span)
    EMAed_dfList_exp2_user_47 = get_EMAed_dfList(raw_dfList_exp2_user_47, span=P.span)

    
    dfList_dict={
                "dfList_exp1": EMAed_dfList_exp1,
                "dfList_exp2": EMAed_dfList_exp2,
                "dfList_exp1_user_47": EMAed_dfList_exp1_user_47,
                "dfList_exp2_user_47": EMAed_dfList_exp2_user_47
    }
    
        

    test_dict_key=scnn_archi_name, P.span
    EER_df_test_dict[test_dict_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, 
                                                                                             exp_config=P, extract_features_func_dict=exp_deep_feature_extractors_dict,
                                                                                             overlap=OVERLAP, best_param_df=rival_test_hyperparameters_df)
    with open(test_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\Top smoothing parameter/s: {test_dict_key}\n")
        f.write(EER_df_test_dict[test_dict_key].to_string())
#-------
#-------
key_column= ["scnn_archi_name", "EMA_span"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

## 3.2 Realworld Approach (per unknown window application of filter)
### Optimizing and Testing

In [ ]:
init_experiment_params(exp_config=P)

exp_deep_archi_extractors_dict=get_exp_deep_feature_extractors_dict(exp_num=EXP_NUM_3_2)
print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "EMA"



preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"

    
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")
    

EER_df_test_dict={}
for scnn_archi_name, exp_deep_feature_extractors_dict in exp_deep_archi_extractors_dict.items():
    
    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{RIVAL_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

    P.span=rival_test_hyperparameters_df["EMA_span"][0]
    P.EMA_per_win_span=P.span
    print(f"EMA span: {P.span}")



    EMAed_dfList_exp2 = get_EMAed_dfList(raw_dfList_exp2, span=P.span)
    
    EMAed_dfList_exp2_user_47 = get_EMAed_dfList(raw_dfList_exp2_user_47, span=P.span)

    
    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": EMAed_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": EMAed_dfList_exp2_user_47
    }
    

    test_dict_key=scnn_archi_name, P.span
    EER_df_test_dict[test_dict_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, 
                                                                                             exp_config=P, 
                                                                                             extract_features_func_dict=exp_deep_feature_extractors_dict,  
                                                                                             overlap=OVERLAP, best_param_df=rival_test_hyperparameters_df)
    with open(test_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\Top smoothing parameter/s: {test_dict_key}\n")
        f.write(EER_df_test_dict[test_dict_key].to_string())
#-------
#-------
key_column= ["scnn_archi_name", "EMA_span"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

# 4. SMA winsize
## 4.1 Naive Approach
### Optimizing and Testing

In [ ]:
init_experiment_params(exp_config=P)

exp_deep_archi_extractors_dict=get_exp_deep_feature_extractors_dict(exp_num=EXP_NUM_4_1)
print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "SMA"



preprocessing_method="Naive"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"
    
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")
    
    

EER_df_test_dict={}
for scnn_archi_name, exp_deep_feature_extractors_dict in exp_deep_archi_extractors_dict.items():
    
    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{RIVAL_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

    P.winsize=rival_test_hyperparameters_df["SMA_winsize"][0]
    print(f"SMA winsize: {P.winsize}")


    SMAed_dfList_exp1 = get_SMAed_dfList(raw_dfList_exp1, winsize=P.winsize)
    SMAed_dfList_exp2 = get_SMAed_dfList(raw_dfList_exp2, winsize=P.winsize)
    
    SMAed_dfList_exp1_user_47 = get_SMAed_dfList(raw_dfList_exp1_user_47, winsize=P.winsize)
    SMAed_dfList_exp2_user_47 = get_SMAed_dfList(raw_dfList_exp2_user_47, winsize=P.winsize)

    
    dfList_dict={
                "dfList_exp1": SMAed_dfList_exp1,
                "dfList_exp2": SMAed_dfList_exp2,
                "dfList_exp1_user_47": SMAed_dfList_exp1_user_47,
                "dfList_exp2_user_47": SMAed_dfList_exp2_user_47
    }
    
    test_dict_key=scnn_archi_name, P.winsize
    EER_df_test_dict[test_dict_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                       extract_features_func_dict=exp_deep_feature_extractors_dict,  overlap=OVERLAP, 
                                                                                       best_param_df=rival_test_hyperparameters_df)
    with open(test_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\Top smoothing parameter/s: {test_dict_key}\n")
        f.write(EER_df_test_dict[test_dict_key].to_string())
#-------
#-------
key_column= ["scnn_archi_name", "SMA_winsize"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

## 4.2 Realworld Approach (per unknown window application of filter)
### Optimizing and Testing

In [ ]:
init_experiment_params(exp_config=P)

exp_deep_archi_extractors_dict=get_exp_deep_feature_extractors_dict(exp_num=EXP_NUM_4_2)
print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "SMA"



preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"

with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")
    
    

EER_df_test_dict={}
for scnn_archi_name, exp_deep_feature_extractors_dict in exp_deep_archi_extractors_dict.items():
    
    rival_test_hyperparameters_file_name=f"{FINAL_EXP_RESULTS_PATH}/{RIVAL_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict_raw_df.json"
    rival_test_hyperparameters_df = pd.read_json(rival_test_hyperparameters_file_name)

    P.winsize=rival_test_hyperparameters_df["SMA_winsize"][0]    
    P.SMA_per_win_winsize=P.winsize
    print(f"SMA winsize: {P.winsize}")


    SMAed_dfList_exp2 = get_SMAed_dfList(raw_dfList_exp2, winsize=P.winsize)
    
    SMAed_dfList_exp2_user_47 = get_SMAed_dfList(raw_dfList_exp2_user_47, winsize=P.winsize)

    
    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": SMAed_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": SMAed_dfList_exp2_user_47
    }
    
    test_dict_key=scnn_archi_name, P.winsize
    EER_df_test_dict[test_dict_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                       extract_features_func_dict=exp_deep_feature_extractors_dict,  overlap=OVERLAP, 
                                                                                       best_param_df=rival_test_hyperparameters_df)
    with open(test_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\Top smoothing parameter/s: {test_dict_key}\n")
        f.write(EER_df_test_dict[test_dict_key].to_string())
#-------
#-------
key_column= ["scnn_archi_name", "SMA_winsize"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

# 5. Butterworth frequency Cut-off + SMA winsize
## 5.1 Naive Approach
### Optimizing and Testin

In [ ]:
init_experiment_params(exp_config=P, exp_num=EXP_NUM_5)

exp_deep_archi_extractors_dict=get_exp_deep_feature_extractors_dict(exp_num=EXP_NUM_5_1)
print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butter+SMA"



preprocessing_method="Naive"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"

with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")
    



EER_df_test_dict={}

for key_pair in tqdm(cut_off_winsize_pairs):
    
    key_pair = tuple(key_pair)
    cut_off_freq, winsize = key_pair[0], key_pair[1]
    P.cut_off_freq=cut_off_freq
    P.winsize=winsize
    print(f"cut_off_freq: {P.cut_off_freq}")
    print(f"winsize: {P.winsize}")

    
    ffted_dfList_exp1 = get_ffted_dfList(raw_dfList_exp1, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    SMAed_dfList_exp1 = get_SMAed_dfList(ffted_dfList_exp1, winsize=P.winsize)
    SMAed_dfList_exp2 = get_SMAed_dfList(ffted_dfList_exp2, winsize=P.winsize)
    
    ffted_dfList_exp1_user_47 = get_ffted_dfList(raw_dfList_exp1_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)

    SMAed_dfList_exp1_user_47 = get_SMAed_dfList(ffted_dfList_exp1_user_47, winsize=P.winsize)
    SMAed_dfList_exp2_user_47 = get_SMAed_dfList(ffted_dfList_exp2_user_47, winsize=P.winsize)
    
    dfList_dict={
                "dfList_exp1": SMAed_dfList_exp1,
                "dfList_exp2": SMAed_dfList_exp2,
                "dfList_exp1_user_47": SMAed_dfList_exp1_user_47,
                "dfList_exp2_user_47": SMAed_dfList_exp2_user_47
    }
    
    EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                       extract_features_func_dict=exp_deep_feature_extractors_dict,  overlap=OVERLAP, 
                                                                                       best_param_df=EER_df_train_dict[min_key])
    with open(test_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\Top smoothing parameter/s: {min_key}\n")
        f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["cut_off_freq", "SMA_winsize"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

## 5.2 Realworld Approach (per unknown window application of filter)
### Optimizing and Testing

In [ ]:
init_experiment_params(exp_config=P, exp_num=EXP_NUM_5)
P.smoothing = "Butter+SMA"

preprocessing_method="Realworld-per_unknown_window"

time_of_execution = time.strftime("%Y-%m-%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/clip={P.scaler_clip}-Smoothing={P.smoothing}-Prep={preprocessing_method}-EER_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/clip={P.scaler_clip}-Smoothing={P.smoothing}-Prep={preprocessing_method}-EER_df_test_dict.txt"

with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{train_file_name}" + "-"*40 + "\n")
    
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")


clf_type="standalone"




for cv_fold_idx in range(len(THREE_FOLD_CV)):
    
    train_set, test_set=THREE_FOLD_CV[cv_fold_idx]
    print(f"train_set: {train_set}")
    print(f"test_set: {test_set}")
    
    # --------------butter33-SMA20----------------------
    P.winsize=20
    P.cut_off_freq=33

    P.Butter_per_win_argdict={
        "filter_order": P.filter_order,
        "cut_off_freq": P.cut_off_freq,
        "sampling_freq": P.sampling_freq,
        "filtfilt": P.filtfilt,
         }   
    P.SMA_per_win_winsize=P.winsize

    print(f"cut_off_freq: {P.cut_off_freq}, winsize: {P.winsize}")




    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    SMAed_dfList_exp2 = get_SMAed_dfList(ffted_dfList_exp2, winsize=P.winsize)

    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    SMAed_dfList_exp2_user_47 = get_SMAed_dfList(ffted_dfList_exp2_user_47, winsize=P.winsize)


    # --------------butter33-SMA20----------------------

    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": SMAed_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": SMAed_dfList_exp2_user_47
    }
        



    exp_deep_archi_extractors_dict=get_exp_deep_feature_extractors_dict(exp_num=EXP_NUM_5_2, 
                                                                        model_architectures=list(TRAINING_CONFIG_CV_DICT["Butter-SMA"][cv_fold_idx]['1000'].keys()), 
                                                                        exp_path_name=EXP_PATH_NAME,
                                                                        window_size_lst=WINDOW_SIZE_LST_NN,
                                                                        variant_suffix=f"Butter33-SMA20-cv{cv_fold_idx}")
        

    key_column= ["cut_off_freq", "SMA_winsize"]
    # -----CV_FOLD-------
    # for cv_fold_idx in range(len(THREE_FOLD_CV)):
    process_cv_fold_OCSVM_LOF_IF_NN(\
                                    cv_fold_idx=cv_fold_idx, 
                                    cv_sets=THREE_FOLD_CV, 
                                    dfList_dict=dfList_dict, 
                                    window_size_lst=WINDOW_SIZE_LST_NN, 
                                    exp_config=P, 
                                    exp_deep_archi_extractors_dict=exp_deep_archi_extractors_dict, 
                                    overlap=OVERLAP, 
                                    param_dist=param_dist, 
                                    train_file_name=train_file_name, 
                                    test_file_name=test_file_name, 
                                    preprocessing_params=(P.cut_off_freq, P.SMA_per_win_winsize),
                                    key_column=key_column,
                                   )
    
    
    del exp_deep_archi_extractors_dict
    gc.collect()

In [ ]:
init_experiment_params(exp_config=P, exp_num=EXP_NUM_5)

exp_deep_archi_extractors_dict=get_exp_deep_feature_extractors_dict(exp_num=EXP_NUM_5_2)
print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")
P.smoothing = "Butter+SMA"



preprocessing_method="Realworld-per_unknown_window"
time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/{P.smoothing}_Mean_EER_{preprocessing_method}_df_test_dict.txt"

with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")




P.Butter_per_win_argdict={
    "filter_order": P.filter_order,
    "cut_off_freq": None,
    "sampling_freq": P.sampling_freq,
    "filtfilt": P.filtfilt,
    }


EER_df_test_dict={}

for key_pair in tqdm(cut_off_winsize_pairs):
    
    key_pair = tuple(key_pair)
    cut_off_freq, winsize = key_pair[0], key_pair[1]
    P.cut_off_freq=cut_off_freq
    P.Butter_per_win_argdict["cut_off_freq"]=P.cut_off_freq
    P.winsize=winsize
    P.SMA_per_win_winsize=P.winsize
    
    print(f"cut_off_freq: {P.cut_off_freq}")
    print(f"winsize: {P.winsize}")

    
    ffted_dfList_exp2 = get_ffted_dfList(raw_dfList_exp2, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    SMAed_dfList_exp2 = get_SMAed_dfList(ffted_dfList_exp2, winsize=P.winsize)
    
    ffted_dfList_exp2_user_47 = get_ffted_dfList(raw_dfList_exp2_user_47, cut_off_freq=P.cut_off_freq, filter_order=P.filter_order, sampling_freq=P.sampling_freq, filtfilt=P.filtfilt)
    SMAed_dfList_exp2_user_47 = get_SMAed_dfList(ffted_dfList_exp2_user_47, winsize=P.winsize)
    
    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": SMAed_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": SMAed_dfList_exp2_user_47
    }
    

    EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                       extract_features_func_dict=exp_deep_feature_extractors_dict,  overlap=OVERLAP, 
                                                                                       best_param_df=EER_df_train_dict[min_key])
    with open(test_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\Top smoothing parameter/s: {min_key}\n")
        f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["cut_off_freq", "SMA_winsize"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

# 6. The effect of Varying Overlap

In [ ]:
init_experiment_params(exp_config=P)

print(f"train_set: {train_set+[47]}")
print(f"test_set: {test_set}")



time_of_execution = time.strftime("%Y/%m/%d-%H:%M:%S")
train_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/overlap_Mean_EER_df_train_dict.txt"
test_file_name=f"{FINAL_EXP_RESULTS_PATH}/{EXP_PATH_NAME}/overlap_Mean_EER_df_test_dict.txt"
with open(train_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{train_file_name}" + "-"*40 + "\n")
    
with open(test_file_name, "w") as f:
    f.write("\n" + "-"*40 + f"{time_of_execution}" + "-"*40 + "\n")
    f.write("\n" + "-"*40 + f"{test_file_name}" + "-"*40 + "\n")
    


EER_df_train_dict={}
EER_df_test_dict={}
for overlap in tqdm(list(range(50, 52))):
    overlap*=0.01
    max_window_size=2000
    step_width = int(max_window_size * (1-overlap))
    max_num_windows=min(len(getIndices(sampleSize=max_window_size, step=step_width, numSamplePoints=P.num_sample_points_per_exp)), 20)
    
    dfList_dict={
                "dfList_exp1": raw_dfList_exp1,
                "dfList_exp2": raw_dfList_exp2,
                "dfList_exp1_user_47": raw_dfList_exp1_user_47,
                "dfList_exp2_user_47": raw_dfList_exp2_user_47
    }
    

    EER_df_train_dict[overlap] = calculate_EER_different_window_sizes_train_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, train_set=train_set, exp_config=P, 
                                                                                     extract_features_func_dict=exp_deep_feature_extractors_dict,  overlap=OVERLAP, 
                                                                                     param_dist=param_dist)
        
    with open(train_file_name, "a") as f:
        f.write("\n" + "-"*22 + "\n")
        f.write(f"\overlap: {overlap}\n")
        f.write(EER_df_train_dict[overlap].to_string())


        
mean_EER_train_dict={}
for key in EER_df_train_dict:
    mean_EER_train_dict[key] = EER_df_train_dict[key]["Mean_EER"].mean()
    
#-------
train_lst = list(mean_EER_train_dict.items())
train_lst.sort(key=lambda i: i[1], reverse=False) #sort ascending as it is an error rate

with open(train_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(train_lst)):
        f.write(f"{i+1}) {train_lst[i]}\n")
        

min_key=train_lst[0][0]
EER_df_test_dict[min_key] = calculate_EER_different_window_sizes_test_OCSVM_IF_LOF(dfList_dict, window_size_lst=WINDOW_SIZE_LST, test_set=test_set, exp_config=P, 
                                                                                   extract_features_func_dict=exp_deep_feature_extractors_dict,  overlap=OVERLAP, 
                                                                                   best_param_df=EER_df_train_dict[min_key])
with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\Top smoothing parameter/s: {min_key}\n")
    f.write(EER_df_test_dict[min_key].to_string())
#-------
#-------
key_column= ["overlap"]
EER_df_test_dict_df=make_raw_exp_df_results(EER_df_test_dict, key_column)
eer_per_window_size_col_df =make_eer_per_window_size_col_df(EER_df_test_dict_df, key_column, window_size_lst=WINDOW_SIZE_LST)

EER_df_test_dict_df.to_json(f'{test_file_name[:-4]}_raw_df.json')
eer_per_window_size_col_df.to_json(f'{test_file_name[:-4]}_eer_per_window_size_col_df.json')
#-------

mean_EER_test_dict={}
for key in EER_df_test_dict:
    mean_EER_test_dict[key] = EER_df_test_dict[key]["Mean_EER"].mean()
    
l = list(mean_EER_test_dict.items())
l.sort(key=lambda i: i[1])

with open(test_file_name, "a") as f:
    f.write("\n" + "-"*22 + "\n")
    f.write(f"\nSorting based on Mean EER among windows\n")
    for i in range(len(l)):
        f.write(f"{i+1}) {l[i]}\n")

In [78]:
exp_deep_archi_extractors_dict=get_exp_deep_feature_extractors_dict(exp_num=EXP_NUM_5_2, 
                                                                    model_architectures=list(TRAINING_CONFIG_CV_DICT["Butter-SMA"][cv_fold_idx]['1000'].keys()), 
                                                                    variant_suffix=f"Butter33-SMA20-cv{cv_fold_idx}")

clip=False_experiments_results/SCNN_trained_models/SCNN-standalone/5.2/SCNN_3_1_conv_1_dense_arg_dict_default/125/SCNN-standalone-5.2-SCNN_3_1_conv_1_dense_arg_dict_default-125-deep_feature_extractor-Butter33-SMA20-cv2
clip=False_experiments_results/SCNN_trained_models/SCNN-standalone/5.2/SCNN_3_1_conv_1_dense_arg_dict_default/250/SCNN-standalone-5.2-SCNN_3_1_conv_1_dense_arg_dict_default-250-deep_feature_extractor-Butter33-SMA20-cv2
clip=False_experiments_results/SCNN_trained_models/SCNN-standalone/5.2/SCNN_3_1_conv_1_dense_arg_dict_default/500/SCNN-standalone-5.2-SCNN_3_1_conv_1_dense_arg_dict_default-500-deep_feature_extractor-Butter33-SMA20-cv2
clip=False_experiments_results/SCNN_trained_models/SCNN-standalone/5.2/SCNN_3_1_conv_1_dense_arg_dict_default/750/SCNN-standalone-5.2-SCNN_3_1_conv_1_dense_arg_dict_default-750-deep_feature_extractor-Butter33-SMA20-cv2
clip=False_experiments_results/SCNN_trained_models/SCNN-standalone/5.2/SCNN_3_1_conv_1_dense_arg_dict_default/1000/SCNN-stan

KeyboardInterrupt: 

In [79]:
del exp_deep_archi_extractors_dict
gc.collect()

792825

# Summary

In [84]:
%run ./Classification_utility-functions.ipynb


SEED: 567
FINAL_EXP_RESULTS_PATH: clip=False_experiments_results
--------------------SEED and CONSTANTS imported--------------------
SEED: 567
Python 3.9.10
EER: 0.333, Threshold: 0.600 <-- Arbitrary case
EER: 0.000, Threshold: 0.900 <-- Best case
EER: 1.000, Threshold: 0.900 <-- Worse case
EER: 0.400, Threshold: 0.200 <-- Worse case
EER: 0.167, Threshold: 0.600 <-- Arbitrary case
EER: 0.000, Threshold: 0.900 <-- Best case
EER: 1.000, Threshold: 0.900 <-- Worse case
EER: 0.333, Threshold: 1.000 <-- Worse case
--------------------Utility functions imported--------------------
SEED: 567
FINAL_EXP_RESULTS_PATH: clip=False_experiments_results
--------------------SEED and CONSTANTS imported--------------------
SEED: 567
Python 3.9.10
--------------------Preprocessing utility functions imported--------------------
SEED: 567
FINAL_EXP_RESULTS_PATH: clip=False_experiments_results
--------------------SEED and CONSTANTS imported--------------------
SEED: 567
Python 3.9.10
--------------------Neu

In [ ]:

df=return_and_save_final_result_df_as_json_NN(final_exp_results_path=FINAL_EXP_RESULTS_PATH, exp_path_name=EXP_PATH_NAME, window_size_lst=WINDOW_SIZE_LST_NN)
df

\
   mean rank                           scnn_archi_name cut_off_freq EMA_span   
0         12    SCNN_1_3_conv_1_dense_arg_dict_default           33        -   
1          5  SCNN_3_123_conv_1_dense_arg_dict_default           33        -   
2          4    SCNN_3_1_conv_1_dense_arg_dict_default           33        -   
3         11    SCNN_3_2_conv_1_dense_arg_dict_default           33        -   
4         10    SCNN_1_3_conv_1_dense_arg_dict_default           33       20   
5          1  SCNN_3_123_conv_1_dense_arg_dict_default           33       20   
6          2    SCNN_3_1_conv_1_dense_arg_dict_default           33       20   
7          3    SCNN_3_2_conv_1_dense_arg_dict_default           33       20   
8          9    SCNN_1_3_conv_1_dense_arg_dict_default           33        -   
9          8  SCNN_3_123_conv_1_dense_arg_dict_default           33        -   
10         7    SCNN_3_1_conv_1_dense_arg_dict_default           33        -   
11         6    SCNN_3_2_conv_1_dense_arg_dict_default           33        -   

                     Window Size                                          \
   SMA_winsize  type         125       250       500       750      1000   
0            -  Real    0.205452  0.129847  0.109420  0.096923  0.089341   
1            -  Real    0.149945  0.102158  0.068907  0.062128  0.074074   
2            -  Real    0.141177  0.099636  0.080076  0.086613  0.047986   
3            -  Real    0.170891  0.147496  0.094217  0.090747  0.069702   
4            -  Real    0.152196  0.116307  0.075605  0.083953  0.081825   
5            -  Real    0.147831  0.086609  0.074547  0.049717  0.053342   
6            -  Real    0.147269  0.100277  0.086894  0.045973  0.048826   
7            -  Real    0.149638  0.097418  0.073418  0.063805  0.060470   
8           20  Real    0.158300  0.108288  0.079527  0.096094  0.066621   
9           20  Real    0.164459  0.102071  0.082405  0.061037  0.062340   
10          20  Real    0.166174  0.095921  0.082035  0.055205  0.068058   
11          20  Real    0.148142  0.105837  0.088246  0.071017  0.053379   

                        
        mean  variance  
0   0.126196  0.002198  
1   0.091442  0.001302  
2   0.091098  0.001145  
3   0.114611  0.001813  
4   0.101977  0.001038  
5   0.082409  0.001568  
6   0.085848  0.001736  
7   0.088950  0.001360  
8   0.101766  0.001250  
9   0.094462  0.001813  
10  0.093479  0.001883  
11  0.093324  0.001320

In [87]:
relative_df=return_and_save_final_relative_result_df_as_json(df, base_case_index=0, final_exp_results_path=FINAL_EXP_RESULTS_PATH, exp_path_name=EXP_PATH_NAME, 
                                                             window_size_lst=WINDOW_SIZE_LST_NN)
relative_df

\
   mean rank                           scnn_archi_name cut_off_freq EMA_span   
0         12    SCNN_1_3_conv_1_dense_arg_dict_default           33        -   
1          5  SCNN_3_123_conv_1_dense_arg_dict_default           33        -   
2          4    SCNN_3_1_conv_1_dense_arg_dict_default           33        -   
3         11    SCNN_3_2_conv_1_dense_arg_dict_default           33        -   
4         10    SCNN_1_3_conv_1_dense_arg_dict_default           33       20   
5          1  SCNN_3_123_conv_1_dense_arg_dict_default           33       20   
6          2    SCNN_3_1_conv_1_dense_arg_dict_default           33       20   
7          3    SCNN_3_2_conv_1_dense_arg_dict_default           33       20   
8          9    SCNN_1_3_conv_1_dense_arg_dict_default           33        -   
9          8  SCNN_3_123_conv_1_dense_arg_dict_default           33        -   
10         7    SCNN_3_1_conv_1_dense_arg_dict_default           33        -   
11         6    SCNN_3_2_conv_1_dense_arg_dict_default           33        -   

                     Window Size                                              \
   SMA_winsize  type         125        250        500        750       1000   
0            -  Real    0.000000   0.000000   0.000000   0.000000   0.000000   
1            -  Real   27.017080  21.324383  37.025407  35.899396  17.087968   
2            -  Real   31.284737  23.266380  26.817830  10.637115  46.289181   
3            -  Real   16.821885 -13.592266  13.894213   6.371153  21.981800   
4            -  Real   25.921547  10.427342  30.904123  13.381505   8.412538   
5            -  Real   28.046282  33.299173  31.870641  48.704231  40.293225   
6            -  Real   28.319630  22.772768  20.586436  52.567220  45.348837   
7            -  Real   27.166325  24.974380  32.902410  34.169388  32.315470   
8           20  Real   22.950557  16.603471  27.319440   0.854582  25.429727   
9           20  Real   19.952819  21.390995  24.689042  37.024943  30.222447   
10          20  Real   19.117797  26.127280  25.027527  43.041756  23.822042   
11          20  Real   27.894897  18.490811  19.350761  26.728271  40.252781   

                          
         mean   variance  
0    0.000000   0.000000  
1   27.539686  40.781573  
2   27.812939  47.892150  
3    9.180698  17.521971  
4   19.191785  52.752776  
5   34.697663  28.647034  
6   31.972869  21.032462  
7   29.514738  38.128851  
8   19.358990  43.121876  
9   25.146544  17.513261  
10  25.925997  14.319715  
11  26.048565  39.935921

In [ ]:
df.style.hide(axis='index').to_latex()


In [ ]:
relative_df.style.hide(axis='index').to_latex()
